In [31]:
import requests
import pandas as pd

def income_api(tick):
	url = "https://last10k-company-v1.p.rapidapi.com/v1/company/income"

	querystring = {"ticker":tick,"formType":"10-k","filingorder":"0"}

	headers = {
		"X-RapidAPI-Key": "8c86128819msh99c25f8c2e98232p12f358jsna52fccb57dbb",
		"X-RapidAPI-Host": "last10k-company-v1.p.rapidapi.com"
	}

	response = requests.get(url, headers=headers, params=querystring)

	print(response.json())

	response_json = response.json()  # Get JSON from response object
	return response_json
	

In [33]:
#ticker_list = ['AAPL','MSFT','AMZN','GOOGL','META','TSLA','NVDA']
ticker_list = ['AAPL','GOOGL']

# Check if 'unique_list' is already defined
if 'incomes' not in globals():
    incomes = {}
    print('created incomes as empty dictionary')

for x in ticker_list:
    if x not in incomes:
        response_json = income_api(tick=x)
        df_merged = parse_income_api(response_json)

    else:
        print((f'{x} is already extracted'))

AAPL is already extracted
GOOGL is already extracted


In [38]:
for x in incomes:
    print(incomes[x])
    #export to CSV
    if x == 'GOOGL':
        df_merged.to_csv('df_merged.csv', index=False)

          company ticker                          statement  \
0   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
1   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
2   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
3   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
4   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
5   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
6   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
7   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
8   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
9   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
10  Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
11  Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
12  Alphabet Inc.  GOOGL  Consolidated Statements Of Income   

         accessionNumber           filingDate               period formType  \
0   0001652044-24-000022  2024-01-31T0

{'meta': {'copyright': 'Copyright 2024 Last10K.com All Rights Reserverd', 'terms': ['https://dev.last10k.com/terms', 'https://www.sec.gov/privacy.htm#security']}, 'data': {'attributes': {'status': 0, 'company': {'name': 'Alphabet Inc.', 'cik': '1652044', 'ticker': 'GOOGL'}, 'filing': {'name': 'Consolidated Statements Of Income', 'accessionNumber': '0001652044-24-000022', 'filingDate': '2024-01-31T00:00:00', 'period': '2023-12-31T00:00:00', 'formType': '10-K', 'filingCount': 0}, 'count': 13, 'result': {'RevenueFromContractWithCustomerExcludingAssessedTax': 307394000000.0, 'CostOfRevenue': 133332000000.0, 'ResearchAndDevelopmentExpense': 45427000000.0, 'SellingAndMarketingExpense': 27917000000.0, 'GeneralAndAdministrativeExpense': 16425000000.0, 'CostsAndExpenses': 223101000000.0, 'OperatingIncomeLoss': 84293000000.0, 'NonoperatingIncomeExpense': 1424000000.0, 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest': 85717000000.0, 'IncomeTaxExpenseBe

In [7]:
"""
import pandas as pd

# Assuming 'response' is your variable holding the API response
# response = api_call()  # Your function to call the API
response_json = response.json()  # Get JSON from response object

# Extract the income statement line items from the JSON
income_statement = response_json['data']['attributes']['result']
print(income_statement)

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(income_statement, orient='index', columns=['Amount'])

# Reset index to make the index a regular column, giving appropriate names
df.reset_index(inplace=True)
df.columns = ['LineItem', 'Amount']

# Optionally, convert 'Amount' to a more readable format (e.g., billions)
#df['Amount'] = df['Amount'] / 1e9  # Convert to billions

print(df)
"""

{'RevenueFromContractWithCustomerExcludingAssessedTax': 307394000000.0, 'CostOfRevenue': 133332000000.0, 'ResearchAndDevelopmentExpense': 45427000000.0, 'SellingAndMarketingExpense': 27917000000.0, 'GeneralAndAdministrativeExpense': 16425000000.0, 'CostsAndExpenses': 223101000000.0, 'OperatingIncomeLoss': 84293000000.0, 'NonoperatingIncomeExpense': 1424000000.0, 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest': 85717000000.0, 'IncomeTaxExpenseBenefit': 11922000000.0, 'NetIncomeLoss': 73795000000.0, 'EarningsPerShareBasic': 5.84, 'EarningsPerShareDiluted': 5.8}
                                             LineItem        Amount
0   RevenueFromContractWithCustomerExcludingAssess...  3.073940e+11
1                                       CostOfRevenue  1.333320e+11
2                       ResearchAndDevelopmentExpense  4.542700e+10
3                          SellingAndMarketingExpense  2.791700e+10
4                     GeneralAndAdministrativeEx

In [30]:
def parse_income_api(response_json):

    # Extract the 'attributes' and 'result' from the JSON
    attributes = response_json['data']['attributes']
    print(attributes)
    company = attributes['company']['name']
    ticker = attributes['company']['ticker']


    parsed_attributes = {
        'company' : company,
        'ticker' : ticker
        }

    # Include the filing attributes into the main attributes dictionary
    filing = attributes.pop('filing')
    for key, value in filing.items():
        parsed_attributes[key] = value  # Add each filing attribute to the main attributes dictionary

    print(parsed_attributes)

    df_att = pd.DataFrame([parsed_attributes])  # The list brackets around the dictionary ensure it is treated as a single row
    df_att.rename(columns={'name': 'statement'}, inplace=True)

    print(df_att)

    # Convert the dictionary to a pandas DataFrame
    income_statement = attributes.pop('result')  # Remove the result and keep the rest of the attributes
    df_pl = pd.DataFrame.from_dict(income_statement, orient='index', columns=['Amount'])

    # Reset index to make the index a regular column, giving appropriate names
    df_pl.reset_index(inplace=True)
    df_pl.columns = ['LineItem', 'Amount']

    print(df_pl)

    # Replicate df_att to match the number of rows in df_pl
    df_att_repeated = pd.concat([df_att]*len(df_pl), ignore_index=True)

    # Concatenate the two DataFrames horizontally
    df_merged = pd.concat([df_att_repeated, df_pl], axis=1)

    print(df_merged)

    #Create a running list
    if ticker not in incomes:
        incomes[ticker] = df_merged
        print((f'Added {ticker} to incomes'))

    print(incomes)



    return df_merged


In [27]:
# # Check if 'unique_list' is already defined
# if 'incomes' not in globals():
#     incomes = {}
#     print('created incomes as empty dictionary')
# #Create a running list
#     if ticker not in incomes:
#         incomes[ticker] = df_merged
#         print(f('Added {ticker} to incomes'))

In [26]:
#export to CSV
#df_merged.to_csv('df_merged.csv', index=False) #, sep='|')
print(incomes)

{'GOOGL':           company ticker                          statement  \
0   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
1   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
2   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
3   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
4   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
5   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
6   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
7   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
8   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
9   Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
10  Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
11  Alphabet Inc.  GOOGL  Consolidated Statements Of Income   
12  Alphabet Inc.  GOOGL  Consolidated Statements Of Income   

         accessionNumber           filingDate               period formType  \
0   0001652044-24-000022  20